# Annosaurus Tutorial

This python3 notebook demonstrates the usage of the [Annosaurus](https://github.com/underwatervideo/annosaurus) API which is used for creating and editing video annotations. To get started you will need to start annosaurus. If you have [Docker](https://www.docker.com/) installed you can spin up annosaurus for testing with:

```
docker run --name=anno -p 8080:8080 hohonuuli/annosaurus
```


### Configure imports and define helper functions

In [414]:
import datetime
import json
import pprint
import random
import requests
import urllib
import uuid

def show(s, data = None):
    pp = pprint.PrettyPrinter(indent=2)
    print("--- " + s)
    if data:
      pp.pprint(data)
    
def iso8601():
    return datetime.datetime.now(datetime.timezone.utc).isoformat()[0:-6] + "Z"
    
def parse_response(r):
    try:
       return json.loads(r.text)
    except:
        s = "URL: %s\n%s (%s): %s" % (r.request.url, r.status_code, r.reason, r.text)
        print(s)
        return {}
    
    
def delete(url):
    return parse_response(requests.delete(url))

def get(url):
    return parse_response(requests.get(url))
    
def post(url, data = {}):
    return parse_response(requests.post(url, data))

def put(url, data = {}):
    return parse_response(requests.put(url, data))


### Define URLs

Normally in an app or script you just define the endpoint and build the other API urls from that.

In [415]:
# Define endpoint. 
endpoint = "http://134.89.9.70:8080"

annotation_url = "%s/v1/annotations" % (endpoint)
image_url = "%s/v1/images" % (endpoint)
image_reference_url = "%s/v1/imagereferences" % (endpoint)
imaged_moment_url = "%s/v1/imagedmoments" % (endpoint)
observation_url = "%s/v1/observations" % (endpoint)
association_url = "%s/v1/associations" % (endpoint)


# High-level APIs

The Annotation and Image APIs are high-level APIs to greatly simplify general usage. They are an abstraction for the lower level _ImagedMoment_, _Observation_, _ImageReference_, and _Association_ APIs. For creating and updating annotations and images, use the highly level APIs. If you need to delete or do fancy stuff you can use the lower level APIs

## Annotation API

The annotation API is used for creating and modifying annotations. You __can not delete__ with this API. Instead, you use the _Observation API_.

Note that the APIs use [UUIDs](https://en.wikipedia.org/wiki/Universally_unique_identifier) as keys to identify particular items. 

### Create

In [416]:
# id to a video file. Typically you get this from your video asset manager. 
# We're just creating a random one to use for this demo.
video_reference_uuid = str(uuid.uuid4())

# Create w/ minimum allowed fields
annotation = post(annotation_url,
                  data = {"video_reference_uuid": video_reference_uuid,
                          "concept": "Nanomia bijuga",
                          "observer": "brian",
                          "recorded_timestamp": "2016-07-28T14:29:01.030Z"})
show("POST: " + annotation_url, annotation)


--- POST: http://134.89.9.70:8080/v1/annotations
{ 'associations': [],
  'concept': 'Nanomia bijuga',
  'image_references': [],
  'imaged_moment_uuid': 'b6dc8ba9-07c2-4257-b5ea-4f8cb192f573',
  'observation_timestamp': '2017-02-16T19:19:39.311Z',
  'observation_uuid': 'db725805-6297-4adc-b36c-8f88dc828b2b',
  'observer': 'brian',
  'recorded_timestamp': '2016-07-28T14:29:01.030Z',
  'video_reference_uuid': 'eb7573be-9ce1-446c-ac05-75781e420db2'}


In [417]:
# Create with all possible fields, including optional values
annotation = post(annotation_url,
                  data = {"video_reference_uuid": video_reference_uuid,      # video or image grouping
                          "concept": "Aegina citrea",                        # Name of what you saw
                          "observer": "schlin",                              # Who made the observation
                          "observation_timestamp": "2016-07-28T15:01:02Z",   # When the observation was make. Default is the servers timestamp
                          "timecode": "01:23:34:09",                         # A tape timecode of annotation
                          "elapsed_time_millis": "112345",                   # Time since start of video of annotation
                          "duration_millis": "1200",                         # How long was object observed
                          "group": "ROV",                                    # A logical group. At MBARI, we might use "ROV", "AUV", "Station M"
                          "activity": "transect",                            # Another logical group. At MBARI, we would use, ascent, descent, transect, cruise, etc.
                          "recorded_timestamp": "2016-07-28T14:39:02.123Z"}) # The time the frame was recorded. e.g. We saw this Aegina on this date.
show("POST: " + annotation_url, annotation)

--- POST: http://134.89.9.70:8080/v1/annotations
{ 'activity': 'transect',
  'associations': [],
  'concept': 'Aegina citrea',
  'duration_millis': 1200,
  'elapsed_time_millis': 112345,
  'group': 'ROV',
  'image_references': [],
  'imaged_moment_uuid': 'a68fdb80-bac0-425c-92c1-020f6adcd07b',
  'observation_timestamp': '2016-07-28T15:01:02Z',
  'observation_uuid': '1c6de7f8-7dcf-4bc5-b753-2b5866a0c877',
  'observer': 'schlin',
  'recorded_timestamp': '2016-07-28T14:39:02.123Z',
  'timecode': '01:23:34:09',
  'video_reference_uuid': 'eb7573be-9ce1-446c-ac05-75781e420db2'}


### Update

In [418]:
# Update/Modify an existing annotation

observation_uuid = annotation["observation_uuid"]

# At a minimum you need the observation_uuid and one field. The observation_timestamp
# will automatically be updated to the time on the server (UTC). Here we just change
# the concept name. Normally, you might need to update the observer field too.
put_url = "%s/%s" % (annotation_url, observation_uuid)
annotation = put(put_url,
                 data = {"observation_uuid": observation_uuid,
                         "concept": "Atolla"})
show("PUT: " + put_url, annotation)

--- PUT: http://134.89.9.70:8080/v1/annotations/1c6de7f8-7dcf-4bc5-b753-2b5866a0c877
{ 'activity': 'transect',
  'associations': [],
  'concept': 'Atolla',
  'duration_millis': 1200,
  'elapsed_time_millis': 112345,
  'group': 'ROV',
  'image_references': [],
  'imaged_moment_uuid': 'a68fdb80-bac0-425c-92c1-020f6adcd07b',
  'observation_timestamp': '2017-02-16T19:19:39.374Z',
  'observation_uuid': '1c6de7f8-7dcf-4bc5-b753-2b5866a0c877',
  'observer': 'schlin',
  'recorded_timestamp': '2016-07-28T14:39:02.123Z',
  'timecode': '01:23:34:09',
  'video_reference_uuid': 'eb7573be-9ce1-446c-ac05-75781e420db2'}


In [419]:
# You can update any and all fields in one call as we do here.
annotation = put(put_url,
                  data = {"video_reference_uuid": str(uuid.uuid4()),         # Here we move the annotation to a new video
                          "concept": "Pandalus platyceros",                  # Name of what you saw
                          "observer": "danelle",                             # Who made the observation
                          "observation_timestamp": iso8601(),                # When the observation was make. Default is the servers timestamp
                          "timecode": "08:00:34:09",                         # A tape timecode of annotation
                          "elapsed_time_millis": "3045999",                  # Time since start of video of annotation
                          "duration_millis": "8",                            # How long was object observed
                          "group": "AUV",                                    # A logical group. At MBARI, we might use "ROV", "AUV", "Station M"
                          "activity": "descent",                             # Another logical group. At MBARI, we would use, ascent, descent, transect, cruise, etc.
                          "recorded_timestamp": "2017-07-28T14:39:02.123Z"}) # The time the frame was recorded. e.g. We saw this Aegina on this date.
show("PUT: " + put_url, annotation)


--- PUT: http://134.89.9.70:8080/v1/annotations/1c6de7f8-7dcf-4bc5-b753-2b5866a0c877
{ 'activity': 'descent',
  'associations': [],
  'concept': 'Pandalus platyceros',
  'duration_millis': 8,
  'elapsed_time_millis': 3045999,
  'group': 'AUV',
  'image_references': [],
  'imaged_moment_uuid': '926fa4f6-368d-47d0-9396-ae00e1a329fa',
  'observation_timestamp': '2017-02-16T19:19:39.405151Z',
  'observation_uuid': '1c6de7f8-7dcf-4bc5-b753-2b5866a0c877',
  'observer': 'danelle',
  'recorded_timestamp': '2017-07-28T14:39:02.123Z',
  'timecode': '08:00:34:09',
  'video_reference_uuid': 'd0a26ac5-5f96-4997-85b2-b28348c4d6c3'}


### Find

In [420]:
# Find an annotation by observation_uuid
get_url = "%s/%s" % (annotation_url, annotation["observation_uuid"])
annotation = get(get_url)
show("GET: " + get_url, annotation)

--- GET: http://134.89.9.70:8080/v1/annotations/1c6de7f8-7dcf-4bc5-b753-2b5866a0c877
{ 'activity': 'descent',
  'associations': [],
  'concept': 'Pandalus platyceros',
  'duration_millis': 8,
  'elapsed_time_millis': 3045999,
  'group': 'AUV',
  'image_references': [],
  'imaged_moment_uuid': '926fa4f6-368d-47d0-9396-ae00e1a329fa',
  'observation_timestamp': '2017-02-16T19:19:39.405Z',
  'observation_uuid': '1c6de7f8-7dcf-4bc5-b753-2b5866a0c877',
  'observer': 'danelle',
  'recorded_timestamp': '2017-07-28T14:39:02.123Z',
  'timecode': '08:00:34:09',
  'video_reference_uuid': 'd0a26ac5-5f96-4997-85b2-b28348c4d6c3'}


In [421]:
# Find all annotation for a specific video
get_url = "%s/videoreference/%s" % (annotation_url, annotation["video_reference_uuid"])
annotations = get(get_url)
show("GET: " + get_url, annotations)

--- GET: http://134.89.9.70:8080/v1/annotations/videoreference/d0a26ac5-5f96-4997-85b2-b28348c4d6c3
[ { 'activity': 'descent',
    'associations': [],
    'concept': 'Pandalus platyceros',
    'duration_millis': 8,
    'elapsed_time_millis': 3045999,
    'group': 'AUV',
    'image_references': [],
    'imaged_moment_uuid': '926fa4f6-368d-47d0-9396-ae00e1a329fa',
    'observation_timestamp': '2017-02-16T19:19:39.405Z',
    'observation_uuid': '1c6de7f8-7dcf-4bc5-b753-2b5866a0c877',
    'observer': 'danelle',
    'recorded_timestamp': '2017-07-28T14:39:02.123Z',
    'timecode': '08:00:34:09',
    'video_reference_uuid': 'd0a26ac5-5f96-4997-85b2-b28348c4d6c3'}]


## Image API

The image API is for registering images. These image can be associated with a particular video but do not have to be. Images are referenced via a URL, not a file path.

### Create

In [422]:
# Although called 'video_reference_uuid', it's a logical group for annotations. 
# For an image set this might just be a random value.
video_reference_uuid = annotation['video_reference_uuid']

# Minimum required fields. Note that one or more indexes are required. I use recorded_timestamp here, 
# but you could also use 'elapased_time_millis' or 'timecode'
image = post(image_url, data = {
    "video_reference_uuid": video_reference_uuid,
    "url": "http://foobar.org/awesomeimage_" + str(random.randint(0, 100000)) + ".jpg",
    "recorded_timestamp": annotation['recorded_timestamp']})
show("POST:" + image_url, image)


--- POST:http://134.89.9.70:8080/v1/images
{ 'elapsed_time_millis': 3045999,
  'height': 0,
  'image_reference_uuid': '018cde12-db99-41ea-9a0a-e2fe0b52ef0b',
  'imaged_moment_uuid': '926fa4f6-368d-47d0-9396-ae00e1a329fa',
  'recorded_timestamp': '2017-07-28T14:39:02.123Z',
  'timecode': '08:00:34:09',
  'url': 'http://foobar.org/awesomeimage_76109.jpg',
  'video_reference_uuid': 'd0a26ac5-5f96-4997-85b2-b28348c4d6c3',
  'width': 0}


In [423]:
# Create with all fields
image = post(image_url, data = {
    "video_reference_uuid": video_reference_uuid,
    "url": "http://foobar.org/onfleekimage_" + str(random.randint(0, 100000)) + ".jpg",
    "recorded_timestamp": iso8601(),
    "timecode": "01:23:45:09",
    "elapsed_time_millis": 123456,
    "width_pixels": 1920,
    "height_pixels": 1080,
    "format": "image/jpg",
    "description": "left-image"})
show("POST: " + image_url , image)

--- POST: http://134.89.9.70:8080/v1/images
{ 'description': 'left-image',
  'elapsed_time_millis': 123456,
  'format': 'image/jpg',
  'height': 1080,
  'image_reference_uuid': 'e3e388a0-7d63-4898-9603-badbb0abb0c1',
  'imaged_moment_uuid': '113aabd2-02b5-45ad-abf6-bc82e9d89b4a',
  'recorded_timestamp': '2017-02-16T19:19:39.533467Z',
  'timecode': '01:23:45:09',
  'url': 'http://foobar.org/onfleekimage_16799.jpg',
  'video_reference_uuid': 'd0a26ac5-5f96-4997-85b2-b28348c4d6c3',
  'width': 1920}


### Update

In [424]:
# This changes all parameters, but you only need to include the ones you want to change
put_url = "%s/%s" % (image_url, image['image_reference_uuid'])
image = put(put_url, data = {
        "video_reference_uuid": str(uuid.uuid4()),
        "url": "http://foobar.org/yodaimage_" + str(random.randint(0, 100000)) + ".tif",
        "recorded_timestamp": iso8601(),
        "timecode": "02:00:15:19",
        "elapsed_time_millis": 666,
        "width_pixels": 2920,
        "height_pixels": 980,
        "format": "image/tiff",
        "description": "right-image"})
show("PUT: " + put_url, image) 

--- PUT: http://134.89.9.70:8080/v1/images/e3e388a0-7d63-4898-9603-badbb0abb0c1
{ 'description': 'right-image',
  'elapsed_time_millis': 666,
  'format': 'image/tiff',
  'height': 980,
  'image_reference_uuid': 'e3e388a0-7d63-4898-9603-badbb0abb0c1',
  'imaged_moment_uuid': '4bc0a97c-cda4-462c-80f4-2c9d8070358e',
  'recorded_timestamp': '2017-02-16T19:19:39.568148Z',
  'timecode': '02:00:15:19',
  'url': 'http://foobar.org/yodaimage_96097.tif',
  'video_reference_uuid': '1b878158-2107-4f31-996b-95a5ace84c13',
  'width': 2920}


### Find

In [425]:
# Find an image by image_reference_uuid
get_url = "%s/%s" %(image_url, image['image_reference_uuid'])
image = get(get_url)
show("GET: " + get_url, image)

--- GET: http://134.89.9.70:8080/v1/images/e3e388a0-7d63-4898-9603-badbb0abb0c1
{ 'description': 'right-image',
  'elapsed_time_millis': 666,
  'format': 'image/tiff',
  'height': 980,
  'image_reference_uuid': 'e3e388a0-7d63-4898-9603-badbb0abb0c1',
  'imaged_moment_uuid': '4bc0a97c-cda4-462c-80f4-2c9d8070358e',
  'recorded_timestamp': '2017-02-16T19:19:39.568Z',
  'timecode': '02:00:15:19',
  'url': 'http://foobar.org/yodaimage_96097.tif',
  'video_reference_uuid': '1b878158-2107-4f31-996b-95a5ace84c13',
  'width': 2920}


In [426]:
# Find all images for a given group (or video)
get_url = "%s/videoreference/%s" % (image_url, video_reference_uuid)
images = get(get_url)
show("GET: " + get_url, images)

--- GET: http://134.89.9.70:8080/v1/images/videoreference/d0a26ac5-5f96-4997-85b2-b28348c4d6c3
[ { 'elapsed_time_millis': 3045999,
    'height': 0,
    'image_reference_uuid': '018cde12-db99-41ea-9a0a-e2fe0b52ef0b',
    'imaged_moment_uuid': '926fa4f6-368d-47d0-9396-ae00e1a329fa',
    'recorded_timestamp': '2017-07-28T14:39:02.123Z',
    'timecode': '08:00:34:09',
    'url': 'http://foobar.org/awesomeimage_76109.jpg',
    'video_reference_uuid': 'd0a26ac5-5f96-4997-85b2-b28348c4d6c3',
    'width': 0}]


In [427]:
# Find image metadata by a URL. Note: you can not use the raw url, urlencode it first
encoded_url = urllib.parse.quote_plus(image['url'])
get_url = "%s/url/%s" % (image_url, encoded_url)
image = get(get_url)
show("GET: " + get_url, image)

--- GET: http://134.89.9.70:8080/v1/images/url/http%3A%2F%2Ffoobar.org%2Fyodaimage_96097.tif
{ 'description': 'right-image',
  'elapsed_time_millis': 666,
  'format': 'image/tiff',
  'height': 980,
  'image_reference_uuid': 'e3e388a0-7d63-4898-9603-badbb0abb0c1',
  'imaged_moment_uuid': '4bc0a97c-cda4-462c-80f4-2c9d8070358e',
  'recorded_timestamp': '2017-02-16T19:19:39.568Z',
  'timecode': '02:00:15:19',
  'url': 'http://foobar.org/yodaimage_96097.tif',
  'video_reference_uuid': '1b878158-2107-4f31-996b-95a5ace84c13',
  'width': 2920}


# Low Level APIs

## API Diagram

![UML Class Diagram](https://github.com/underwatervideo/annosaurus/blob/master/src/site/images/annosaurus_classes.png)

## Association API

Associations are extra descriptions that you can attach to your annotation. Things like color, comments, resting upon, position in the image, a measurement, etc. Associations have the form 'link name | to concept | link value'. 

- link name: Tells you what the association contains. e.g. 'comment' or 'upon substrate'
- to concept: Indicates are relation. For example if your _link name_ is `eating` the _to concept_ indicates what it's eating. e.g. 'Squid'. When referring to itself, the custom is to use a _to concept_ of `self`. (self is the default if you don't supply a _to concept_. e.g `eating | squid | nil`
- link value: A value for the association. The default value is `nil`. Some examples:
  - `population quantity | self | 12`
  - `surface color | self | red`
  - `distance measurement | self | {"image_reference_uuid": "acc435...", "x": [100, 234], "y": [34 1200], "comment": "dorsal spine"}`
  
A final optional parameter is the mimetype of the _link value_. The default is `text/plain`, but it could `application/json` or whatever your specific application needs

### Create

In [428]:
# Create with minimum required parameters
association = post(association_url, data = {
    "observation_uuid": annotation['observation_uuid'],
    "link_name": "swimming"})
show("POST: " + association_url, association)

--- POST: http://134.89.9.70:8080/v1/associations
{ 'last_updated_time': '2017-02-16T11:19:39Z',
  'link_name': 'swimming',
  'link_value': 'nil',
  'mime_type': 'text/plain',
  'to_concept': 'self',
  'uuid': '75552ae2-96d0-472d-b8c2-551aa0e3b57c'}


In [429]:
# Create with all possible parameters
association = post(association_url, data = {
    "observation_uuid": annotation['observation_uuid'],
    "link_name": "distance measurement", 
    "to_concept": "self",
    "link_value": '{"image_reference_uuid":' + image['image_reference_uuid'] + ', "x": [100, 234], "y": [34 1200], "comment": "dorsal spine"}',
    "mime_type": "application/json"})
show("POST: " + association_url, association)

--- POST: http://134.89.9.70:8080/v1/associations
{ 'last_updated_time': '2017-02-16T11:19:39Z',
  'link_name': 'distance measurement',
  'link_value': '{"image_reference_uuid":e3e388a0-7d63-4898-9603-badbb0abb0c1, '
                '"x": [100, 234], "y": [34 1200], "comment": "dorsal spine"}',
  'mime_type': 'application/json',
  'to_concept': 'self',
  'uuid': '4cbb2545-855c-4424-b6dd-699b7d2905a0'}


### Update

In [430]:
# Modify an existing association. We use all fields here but normally
# just include the ones you are changing
put_url = "%s/%s" % (association_url, association['uuid'])
association = put(put_url, data = {
    "observation_uuid": annotation["observation_uuid"],
    "link_name": "eating",
    "to_concept": "Cranchia scabra",
    "link_value": "nil",
    "mime_type": "text/plain"})
show("PUT: " + put_url, association)

--- PUT: http://134.89.9.70:8080/v1/associations/4cbb2545-855c-4424-b6dd-699b7d2905a0
{ 'last_updated_time': '2017-02-16T11:19:39Z',
  'link_name': 'eating',
  'link_value': 'nil',
  'mime_type': 'text/plain',
  'to_concept': 'Cranchia scabra',
  'uuid': '4cbb2545-855c-4424-b6dd-699b7d2905a0'}


In [431]:
# Special method to change ALL to_concepts in the data store. Useful when you've changed
# a species name and need to update all data
put_url = "%s/toconcept/rename" % (association_url)
r = put(put_url, data = {
        "old": "Cranchia scabra",
        "new": "Taonius borealis"})
show("PUT: " + put_url, r)

--- PUT: http://134.89.9.70:8080/v1/associations/toconcept/rename
{ 'new_concept': 'Taonius borealis',
  'number_updated': '0',
  'old_concept': 'Cranchia scabra'}


### Find

In [432]:
# Find by uuid
get_url = "%s/%s" % (association_url, association['uuid'])
association = get(get_url)
show("GET: " + get_url, association)

--- GET: http://134.89.9.70:8080/v1/associations/4cbb2545-855c-4424-b6dd-699b7d2905a0
{ 'last_updated_time': '2017-02-16T11:19:39Z',
  'link_name': 'eating',
  'link_value': 'nil',
  'mime_type': 'text/plain',
  'to_concept': 'Cranchia scabra',
  'uuid': '4cbb2545-855c-4424-b6dd-699b7d2905a0'}


In [433]:
# Find all associations in a video or image group with a given link_name
get_url = "%s/%s/%s" % (association_url, annotation['video_reference_uuid'], "eating")
associations = get(get_url)
show("GET: " + get_url, associations)

--- GET: http://134.89.9.70:8080/v1/associations/d0a26ac5-5f96-4997-85b2-b28348c4d6c3/eating
[ { 'last_updated_time': '2017-02-16T11:19:39Z',
    'link_name': 'eating',
    'link_value': 'nil',
    'mime_type': 'text/plain',
    'to_concept': 'Cranchia scabra',
    'uuid': '4cbb2545-855c-4424-b6dd-699b7d2905a0'}]


In [434]:
# Count the number of usages of a particular to_concept.
get_url = "%s/toconcept/count/%s" % (association_url, "Taonius borealis")
r = get(get_url)
show("GET: " + get_url, r)

--- GET: http://134.89.9.70:8080/v1/associations/toconcept/count/Taonius borealis
{'concept': 'Taonius borealis', 'count': '0'}


### Delete

In [435]:
delete_url = "%s/%s" % (association_url, association['uuid'])
delete(delete_url)

URL: http://134.89.9.70:8080/v1/associations/4cbb2545-855c-4424-b6dd-699b7d2905a0
204 (Success! Deleted association with UUID of 4cbb2545-855c-4424-b6dd-699b7d2905a0): 


{}

## ImagedMoment API

All images and annotations must be related back to real-world time AND/OR a moment in a video. This is represented by an imaged moment which contains:

- timecode
- elapsed time (since the start of a video)
- recorded timestamp (the time that an image or video frame was captured

When you set one of these indicies using the Annotation or Image API the appropriate imaged moment is created (if needed) or the data will be moved to an existing imaged moment. You can __not__ create an ImagedMoment directly with this API.

### Find

In [436]:
# Find all (by default the limit and offset are 1000 and 0 repspectively)
imaged_moments = get(imaged_moment_url)
show("GET: " + imaged_moment_url, imaged_moments)


--- GET: http://134.89.9.70:8080/v1/imagedmoments
[ { 'image_references': [],
    'last_updated_time': '2017-02-16T11:18:30Z',
    'observations': [ { 'associations': [],
                        'concept': 'Nanomia bijuga',
                        'last_updated_time': '2017-02-16T11:18:30Z',
                        'observation_timestamp': '2017-02-16T19:18:28.775Z',
                        'observer': 'brian',
                        'uuid': '38040f18-1033-4696-9bc7-81536a37122c'}],
    'recorded_date': '2016-07-28T14:29:01.030Z',
    'uuid': '12edbf8d-d539-4ee0-a022-9d3d4985ba69',
    'video_reference_uuid': '007cb3e1-5a2e-498f-876e-cb355334122b'},
  { 'elapsed_time': 3045999,
    'image_references': [ { 'height_pixels': 0,
                            'last_updated_time': '2017-02-16T11:18:30Z',
                            'url': 'http://foobar.org/awesomeimage_2198.jpg',
                            'uuid': '6278f84d-371d-428c-9bb8-63685d172b9a',
                            'width_pi

In [437]:
# Find all using explicit limit and offset
get_url = "%s?limit=2&offset=0" % (imaged_moment_url)
imaged_moments = get(get_url)
show("GET: " + get_url, imaged_moments)

--- GET: http://134.89.9.70:8080/v1/imagedmoments?limit=2&offset=0
[ { 'image_references': [],
    'last_updated_time': '2017-02-16T11:18:30Z',
    'observations': [ { 'associations': [],
                        'concept': 'Nanomia bijuga',
                        'last_updated_time': '2017-02-16T11:18:30Z',
                        'observation_timestamp': '2017-02-16T19:18:28.775Z',
                        'observer': 'brian',
                        'uuid': '38040f18-1033-4696-9bc7-81536a37122c'}],
    'recorded_date': '2016-07-28T14:29:01.030Z',
    'uuid': '12edbf8d-d539-4ee0-a022-9d3d4985ba69',
    'video_reference_uuid': '007cb3e1-5a2e-498f-876e-cb355334122b'},
  { 'elapsed_time': 3045999,
    'image_references': [ { 'height_pixels': 0,
                            'last_updated_time': '2017-02-16T11:18:30Z',
                            'url': 'http://foobar.org/awesomeimage_2198.jpg',
                            'uuid': '6278f84d-371d-428c-9bb8-63685d172b9a',
                    

In [438]:
# Find one by its UUID
get_url = "%s/%s" % (imaged_moment_url, imaged_moments[0]['uuid'])
imaged_moment = get(get_url)
show("GET: "+ get_url, imaged_moment)

--- GET: http://134.89.9.70:8080/v1/imagedmoments/12edbf8d-d539-4ee0-a022-9d3d4985ba69
{ 'image_references': [],
  'last_updated_time': '2017-02-16T11:18:30Z',
  'observations': [ { 'associations': [],
                      'concept': 'Nanomia bijuga',
                      'last_updated_time': '2017-02-16T11:18:30Z',
                      'observation_timestamp': '2017-02-16T19:18:28.775Z',
                      'observer': 'brian',
                      'uuid': '38040f18-1033-4696-9bc7-81536a37122c'}],
  'recorded_date': '2016-07-28T14:29:01.030Z',
  'uuid': '12edbf8d-d539-4ee0-a022-9d3d4985ba69',
  'video_reference_uuid': '007cb3e1-5a2e-498f-876e-cb355334122b'}


In [439]:
# Find all video_reference_uuids used in the entire database
get_url = "%s/videoreference" % (imaged_moment_url)
vrs = get(get_url)
show("GET: " + get_url, vrs)

--- GET: http://134.89.9.70:8080/v1/imagedmoments/videoreference
{ 'values': [ '007cb3e1-5a2e-498f-876e-cb355334122b',
              '1b878158-2107-4f31-996b-95a5ace84c13',
              '8ab8197f-6b04-4a05-93f6-434f26a4b006',
              'd0a26ac5-5f96-4997-85b2-b28348c4d6c3',
              'eb7573be-9ce1-446c-ac05-75781e420db2',
              'ed0e2101-5e03-4f78-bc41-235c60fc6a69']}


In [440]:
# Find video_reference_uuids used in the database but limit with limit and offset
get_url = "%s/videoreference?limit=2&offset=0" % (imaged_moment_url)
vrs = get(get_url)
show("GET: " + get_url, vrs)

--- GET: http://134.89.9.70:8080/v1/imagedmoments/videoreference?limit=2&offset=0
{ 'values': [ '007cb3e1-5a2e-498f-876e-cb355334122b',
              '1b878158-2107-4f31-996b-95a5ace84c13']}


In [441]:
# Find all imaged moments for a given video
get_url = "%s/videoreference/%s" % (imaged_moment_url, vrs['values'][0])
imaged_moments = get(get_url)
show("GET: " + get_url, imaged_moments)

--- GET: http://134.89.9.70:8080/v1/imagedmoments/videoreference/007cb3e1-5a2e-498f-876e-cb355334122b
[ { 'image_references': [],
    'last_updated_time': '2017-02-16T11:18:30Z',
    'observations': [ { 'associations': [],
                        'concept': 'Nanomia bijuga',
                        'last_updated_time': '2017-02-16T11:18:30Z',
                        'observation_timestamp': '2017-02-16T19:18:28.775Z',
                        'observer': 'brian',
                        'uuid': '38040f18-1033-4696-9bc7-81536a37122c'}],
    'recorded_date': '2016-07-28T14:29:01.030Z',
    'uuid': '12edbf8d-d539-4ee0-a022-9d3d4985ba69',
    'video_reference_uuid': '007cb3e1-5a2e-498f-876e-cb355334122b'}]


In [442]:
# Find an imaged moment by one of the observations it contains
get_url = "%s/observation/%s" % (imaged_moment_url, annotation['observation_uuid'])
imaged_moment = get(get_url)
show("GET: " + get_url, imaged_moment)

--- GET: http://134.89.9.70:8080/v1/imagedmoments/observation/1c6de7f8-7dcf-4bc5-b753-2b5866a0c877
{ 'elapsed_time': 3045999,
  'image_references': [ { 'height_pixels': 0,
                          'last_updated_time': '2017-02-16T11:19:39Z',
                          'url': 'http://foobar.org/awesomeimage_76109.jpg',
                          'uuid': '018cde12-db99-41ea-9a0a-e2fe0b52ef0b',
                          'width_pixels': 0}],
  'last_updated_time': '2017-02-16T11:19:39Z',
  'observations': [ { 'activity': 'descent',
                      'associations': [ { 'last_updated_time': '2017-02-16T11:19:39Z',
                                          'link_name': 'swimming',
                                          'link_value': 'nil',
                                          'mime_type': 'text/plain',
                                          'to_concept': 'self',
                                          'uuid': '75552ae2-96d0-472d-b8c2-551aa0e3b57c'}],
                      'co

### Update

In [443]:
# We show all possible fields here, but you only need to include the ones you change
put_url = "%s/%s" % (imaged_moment_url, imaged_moment['uuid'])
imaged_moment = put(put_url, data = {
    "timecode": "11:22:33:00",
    "elapsed_time": 100,
    "recorded_timestamp": iso8601(),
    "video_reference_uuid": str(uuid.uuid4())})
show("PUT: " + put_url, imaged_moment)

--- PUT: http://134.89.9.70:8080/v1/imagedmoments/926fa4f6-368d-47d0-9396-ae00e1a329fa
{ 'elapsed_time': 3045999,
  'image_references': [ { 'height_pixels': 0,
                          'last_updated_time': '2017-02-16T11:19:39Z',
                          'url': 'http://foobar.org/awesomeimage_76109.jpg',
                          'uuid': '018cde12-db99-41ea-9a0a-e2fe0b52ef0b',
                          'width_pixels': 0}],
  'last_updated_time': '2017-02-16T11:19:40Z',
  'observations': [ { 'activity': 'descent',
                      'associations': [ { 'last_updated_time': '2017-02-16T11:19:39Z',
                                          'link_name': 'swimming',
                                          'link_value': 'nil',
                                          'mime_type': 'text/plain',
                                          'to_concept': 'self',
                                          'uuid': '75552ae2-96d0-472d-b8c2-551aa0e3b57c'}],
                      'concept': 'Pan

### Delete

Be aware that deleting an imaged moment deletes it, all the observations at that moment, all image references at that moment and all ancillary data at that moment. Be sure that's really what you want to do.

In [444]:
delete_url = "%s/%s" % (imaged_moment_url, imaged_moment['uuid'])
delete(delete_url)

URL: http://134.89.9.70:8080/v1/imagedmoments/926fa4f6-368d-47d0-9396-ae00e1a329fa
204 (Success! Deleted ImagedMoment with UUID of 926fa4f6-368d-47d0-9396-ae00e1a329fa): 


{}

## Observation API

Observations are the individual annotations. Thsi API has a number of find operations, an update one (although you can use the annotation api for that) and allows you to delete individual observations



In [445]:
# House keeping. Since we're deleting stuff, let's make sure we have something to work with
# Create an new annotation
annotation = post(annotation_url,
                  data = {"video_reference_uuid": str(uuid.uuid4()),
                          "concept": "Nanomia bijuga",
                          "observer": "brian",
                          "recorded_timestamp": iso8601()})

association = post(association_url, data = {
    "observation_uuid": annotation['observation_uuid'],
    "link_name": "swimming"})

image = post(image_url, data = {
    "video_reference_uuid": annotation['video_reference_uuid'],
    "url": "http://foobar.org/awesomeimage_" + str(random.randint(0, 100000)) + ".jpg",
    "recorded_timestamp": annotation['recorded_timestamp']})


In [446]:
### Find

In [447]:
# Find an observation by its uuid
get_url = "%s/%s" % (observation_url, annotation['observation_uuid'])
observation = get(get_url)
show("GET: " + get_url, observation)

--- GET: http://134.89.9.70:8080/v1/observations/ba2339d1-e75a-4be8-89af-481c6f54de98
{ 'associations': [ { 'last_updated_time': '2017-02-16T11:19:40Z',
                      'link_name': 'swimming',
                      'link_value': 'nil',
                      'mime_type': 'text/plain',
                      'to_concept': 'self',
                      'uuid': '14d0be40-7224-4c18-a00a-dd050e53367e'}],
  'concept': 'Nanomia bijuga',
  'last_updated_time': '2017-02-16T11:19:40Z',
  'observation_timestamp': '2017-02-16T19:19:40.217Z',
  'observer': 'brian',
  'uuid': 'ba2339d1-e75a-4be8-89af-481c6f54de98'}


In [448]:
# Find observations by video
get_url = "%s/videoreference/%s" % (observation_url, annotation['video_reference_uuid'])
observations = get(get_url)
show("GET: " + get_url, observations)

--- GET: http://134.89.9.70:8080/v1/observations/videoreference/7de4759f-9eab-4988-83e3-ad58f3dac9b1
[ { 'associations': [ { 'last_updated_time': '2017-02-16T11:19:40Z',
                        'link_name': 'swimming',
                        'link_value': 'nil',
                        'mime_type': 'text/plain',
                        'to_concept': 'self',
                        'uuid': '14d0be40-7224-4c18-a00a-dd050e53367e'}],
    'concept': 'Nanomia bijuga',
    'last_updated_time': '2017-02-16T11:19:40Z',
    'observation_timestamp': '2017-02-16T19:19:40.217Z',
    'observer': 'brian',
    'uuid': 'ba2339d1-e75a-4be8-89af-481c6f54de98'}]


In [449]:
# Find by an observation by an association it contains
get_url = "%s/association/%s" % (observation_url, association['uuid'])
observation = get(get_url)
show("GET: " + get_url, observation)

--- GET: http://134.89.9.70:8080/v1/observations/association/14d0be40-7224-4c18-a00a-dd050e53367e
{ 'associations': [ { 'last_updated_time': '2017-02-16T11:19:40Z',
                      'link_name': 'swimming',
                      'link_value': 'nil',
                      'mime_type': 'text/plain',
                      'to_concept': 'self',
                      'uuid': '14d0be40-7224-4c18-a00a-dd050e53367e'}],
  'concept': 'Nanomia bijuga',
  'last_updated_time': '2017-02-16T11:19:40Z',
  'observation_timestamp': '2017-02-16T19:19:40.217Z',
  'observer': 'brian',
  'uuid': 'ba2339d1-e75a-4be8-89af-481c6f54de98'}


In [450]:
# Find all concepts used on all annotations
get_url = "%s/concepts" % (observation_url)
concepts = get(get_url)
show("GET: " + get_url, concepts)

--- GET: http://134.89.9.70:8080/v1/observations/concepts
{'values': ['Nanomia bijuga', 'Pandalus platyceros']}


In [451]:
# Find all concepts used when annotating a particular video or image group
get_url = "%s/concepts/%s" % (observation_url, annotation['video_reference_uuid'])
concepts = get(get_url)
show("GET: " + get_url, concepts)

--- GET: http://134.89.9.70:8080/v1/observations/concepts/7de4759f-9eab-4988-83e3-ad58f3dac9b1
{'values': ['Nanomia bijuga']}


### Update

In [452]:
# Update with all fields. Normally, just include the fields that you want to change
put_url = "%s/%s" % (observation_url, annotation['observation_uuid'])
observation = put(put_url, data = {
    "concept": "Teuthoidea",
    "observer": "Barack Obama",
    "observation_timestamp": iso8601(),
    "duration": 1000,
    "group": "AUV Dorado",
    "activity": "descent"})
# There's also an imaged_moment_uuid field if you want to move an observation to a different moment.
# I didn't include it because of time and more complicated setup.

show("PUT: " + put_url, observation)

--- PUT: http://134.89.9.70:8080/v1/observations/ba2339d1-e75a-4be8-89af-481c6f54de98
{ 'activity': 'descent',
  'associations': [ { 'last_updated_time': '2017-02-16T11:19:40Z',
                      'link_name': 'swimming',
                      'link_value': 'nil',
                      'mime_type': 'text/plain',
                      'to_concept': 'self',
                      'uuid': '14d0be40-7224-4c18-a00a-dd050e53367e'}],
  'concept': 'Teuthoidea',
  'group': 'AUV Dorado',
  'last_updated_time': '2017-02-16T11:19:40Z',
  'observation_timestamp': '2017-02-16T19:19:40.444708Z',
  'observer': 'Barack Obama',
  'uuid': 'ba2339d1-e75a-4be8-89af-481c6f54de98'}


In [453]:
### Delete

In [454]:
# Note that if an imaged moment only contains the observation you are deleting
# the imaged_moment will be deleted to.
delete_url = "%s/%s" % (observation_url, annotation['observation_uuid'])
delete(delete_url)

URL: http://134.89.9.70:8080/v1/observations/ba2339d1-e75a-4be8-89af-481c6f54de98
204 (Success! Deleted observation with UUID of ba2339d1-e75a-4be8-89af-481c6f54de98): 


{}

## ImageReference API

ImageReference is a pointer to an actual image. In general, you use the Image API to create and update them. You will need to use the ImageReference API to delete them though

### Update

In [455]:
# As before we show all params you can change. But you only have to provide the ones you 
# are actually changing
put_url = "%s/%s" % (image_reference_url, image['image_reference_uuid'])
image_reference = put(put_url, data = {
    "url": "http://foobar.org/vaderimage_" + str(random.randint(0, 100000)) + ".webp",
    "format": "image/webp",
    "width_pixels": 5000,
    "height_pixels": 3000,
    "description": "Tripod-mounted camera image",
    "imaged_moment_uuid": annotation['imaged_moment_uuid']
    })
show("PUT: " + put_url, image_reference)

--- PUT: http://134.89.9.70:8080/v1/imagereferences/b72a54b6-62b3-404e-a6b9-fa788c4f9e4e
{ 'description': 'Tripod-mounted camera image',
  'format': 'image/webp',
  'height_pixels': 3000,
  'last_updated_time': '2017-02-16T11:19:40Z',
  'url': 'http://foobar.org/vaderimage_95045.webp',
  'uuid': 'b72a54b6-62b3-404e-a6b9-fa788c4f9e4e',
  'width_pixels': 5000}


### Find

In [456]:
get_url = "%s/%s" % (image_reference_url, image_reference['uuid'])
image_reference = get(get_url)
show("GET: " + get_url, image_reference)


--- GET: http://134.89.9.70:8080/v1/imagereferences/b72a54b6-62b3-404e-a6b9-fa788c4f9e4e
{ 'description': 'Tripod-mounted camera image',
  'format': 'image/webp',
  'height_pixels': 3000,
  'last_updated_time': '2017-02-16T11:19:40Z',
  'url': 'http://foobar.org/vaderimage_95045.webp',
  'uuid': 'b72a54b6-62b3-404e-a6b9-fa788c4f9e4e',
  'width_pixels': 5000}


### Delete

In [457]:
delete_url = "%s/%s" % (image_reference_url, image_reference['uuid'])
delete(delete_url)

URL: http://134.89.9.70:8080/v1/imagereferences/b72a54b6-62b3-404e-a6b9-fa788c4f9e4e
204 (Success! Deleted observation with UUID of b72a54b6-62b3-404e-a6b9-fa788c4f9e4e): 


{}